In [1]:
import logging
import requests
import pandas as pd 
from datetime import datetime
from datetime import timedelta 
import json
import os.path
import time

def getJson(dateIn=None):
    if dateIn == None:
        current_date = datetime.today()
    else:
        current_date = dateIn
    days_subtraction = timedelta(1)
    day_add = timedelta(1)
    reference_date = current_date - days_subtraction + days_subtraction
    end_date = reference_date + day_add
    reference_date = reference_date.strftime('%Y-%m-%d')
    print("Processing:",reference_date)
    end_date = end_date.strftime('%Y-%m-%d')
    # print(reference_date)


    # set up the link to obtain information for the last 7 days
    link_part_start = 'https://www-integration.find-tender.service.gov.uk/api/1.0/ocdsReleasePackages?updatedFrom='
    link_part_end = 'T00:00:00'
    link = link_part_start + reference_date + link_part_end + "&limit=100&updatedTo="+end_date +link_part_end

    # Use get request to call the API
    print("Starting scraping with",link)
    response = requests.get(link)

    # convert string to dict
    
    response = response.json()
    #response = response.replace("Infinity", " -99 " )
    data = response["releases"]
    if "links" in response.keys():
        nextpage = response['links']['next']
        
        # get the data from all the pages from the API
        print("Starting the loop")
        #print(response.keys())
    while 1:
        try: 
            print("Going for "+nextpage)
            A = updateData(data, nextpage)
            #print("A = ",A)
            if A == 0:
                print("Case 0: Items found: ",len(data))
                data = json.dumps(data).replace("Infinity", " -99 " )
                
                with open("cache/"+reference_date+'.json', 'w') as outfile:
                    outfile.write(data)
                return data
            else:
                print("New round")
                data, nextpage = A
        except:
            print("The data is now complete")
            print("Items found: ",len(data))
            
            data = json.dumps(data).replace("Infinity", " -99 " )
            with open("cache/"+reference_date+'.json', 'w') as outfile:
                outfile.write(data)
            return data

def updateData(data, nextpage):
    print("NextPage: "+nextpage)
 
    response_next_page = requests.get(nextpage)
    data_next_page = response_next_page.json()
    print ("Old page contains",len(data)," - New page contains ",len(data_next_page["releases"]),"elements.")
    for newLine in data_next_page["releases"]:
        data.append(newLine) 
    nextpage = data_next_page['links']['next']
    print("New: "+nextpage)
    return data, nextpage
    

In [3]:
for k in range(50):
    start = (datetime.today()-timedelta(k))
    fn = start.strftime('%Y-%m-%d')
    print(fn)
    if os.path.isfile('cache/'+fn+".json"):
        print(fn,"exists")
    else:
        print('cache/'+fn+".json","does not exist")
        a = getJson(dateIn=start) 
    time.sleep(0.5)

2022-04-07
cache/2022-04-07.json does not exist
Processing: 2022-04-07
Starting scraping with https://www-integration.find-tender.service.gov.uk/api/1.0/ocdsReleasePackages?updatedFrom=2022-04-07T00:00:00&limit=100&updatedTo=2022-04-08T00:00:00
The data is now complete
Items found:  30
2022-04-06
cache/2022-04-06.json does not exist
Processing: 2022-04-06
Starting scraping with https://www-integration.find-tender.service.gov.uk/api/1.0/ocdsReleasePackages?updatedFrom=2022-04-06T00:00:00&limit=100&updatedTo=2022-04-07T00:00:00
The data is now complete
Items found:  48
2022-04-05
cache/2022-04-05.json does not exist
Processing: 2022-04-05
Starting scraping with https://www-integration.find-tender.service.gov.uk/api/1.0/ocdsReleasePackages?updatedFrom=2022-04-05T00:00:00&limit=100&updatedTo=2022-04-06T00:00:00
The data is now complete
Items found:  31
2022-04-04
cache/2022-04-04.json does not exist
Processing: 2022-04-04
Starting scraping with https://www-integration.find-tender.service.g

KeyboardInterrupt: 